In [13]:
import numpy as np

In [14]:
# swap rows
def swap_rows(A,i,j):
    '''
    Interchange rows i and j of a matrix A
    Note the identity matrix E is always a square matrix
    Swap rows of identity matrix E to interchange rows of A
    Returns E @ A
    '''
    n = A.shape[0]
    E = np.eye(n)
    E[i,i] = 0
    E[j,j] = 0
    E[i,j] = 1
    E[j,i] = 1
    return E @ A

In [15]:
# add rows
def add_rows(A,m,i,j):
    
    '''
    m: multiplier
    Add m times row j to row i in a matrix A
    '''
    # n stores the the size of a matrix A of size n x n
    n = A.shape[0] # row dimension of A
    E = np.eye(n) # nxn identity matrix
    if i==j:
        E[i,j] = m + 1
    else:
        E[i,j] = m
    return E @ A

In [16]:
# scale row
def scale_row(A,s,i):
    '''
    A: a matrix 
    s: scale factor
    Multiply row i of A by scale factor s
    '''
    n = A.shape[0]
    E = np.eye(n)
# [i,i] is the index of diagonal element of row i that is scaled by s
    E[i,i] = s
    return E @ A

In [17]:
# Bisection Method
# Given : f, a, b, tol

import numpy as np

def bisection_method(f, a, b, tol):
    if np.sign(f(a)) == np.sign(f(b)):
        print("No root in the given interval")
        return None

    start = a
    end = b

    for i in range(500):
        mid = start + (end - start) / 2

        if np.sign(f(mid)) == 0 or np.abs(f(mid)) <= tol:
            print("Root found")
            return mid

        if np.sign(f(mid)) != np.sign(f(start)):
            end = mid
        else:
            start = mid

    print("Maximum number of iterations reached. No root found.")
    return None

def f(x):
    return x**3 - 4

result = bisection_method(f, -10, 10, 0.0001)
print("Root:", np.round(result,4))


Root found
Root: 1.5874


In [18]:
# Newton-Raphson Method
# Given : f, df, x0, tol
# Formula : x1 = x0 - f(x0)/f'(x0)

def newton_raphson(f, df, x0, tol):
    for i in range(500):
        x1 = x0 - f(x0)/df(x0)
        if np.abs(f(x1)) <= tol:
            print("Root found")
            return x1
        x0 = x1
    print("Maximum number of iterations reached. No root found.")
    return None    

def f(x):
    return x**3 - 4

def df(x):
    return 3*(x**2)

result = newton_raphson(f, df, 1, 0.0001)
print("Root:", np.round(result,4))

Root found
Root: 1.5874


In [19]:
# Secant Method
# Given : f, x0, x1, tol
# Formula : x2 = x1 - f(x1)(x1 - x0)/(f'(x1) - f'(x0))

def secant_method(f, x0, x1, tol):
    for i in range(500):
        if f(x0) == f(x1):
            print('Divide by zero error!')
            break

        x2 = x1 - f(x1)*((x1 - x0)/(f(x1) - f(x0)))

        if np.abs(f(x2)) <= tol:
            print("Root found")
            return x2
        x0 = x1
        x1 = x2
    print("Maximum number of iterations reached. No root found.")
    return None

def f(x):
    return x**3 - 4

result = secant_method(f, 1, 100, 0.0001)
print("Root:", np.round(result,4))

Root found
Root: 1.5874


In [20]:
# Gauss Methods

In [86]:
def partial_pivoting_method(mat):
    shape = np.shape(mat)
    # matrix of m x n
    m = shape[0]
    n = shape[1]
    max_val_row = [0]* m
    
    #determining maximum value contain rows for each column
    for i in range(m):
        max_val_row[i] = i
        for j in range(i,n-1,1):
            if abs(mat[j][i]) > abs(mat[i][max_val_row[i]]) :
                max_val_row[i] = j

    cnt = 0 # number of row interchanges
    #here we are swaping rows with the max value rows and then pivoting(column by column)
    for i in range(m):
        if max_val_row[i] != i:
            mat = swap_rows(mat, i, max_val_row[i])
            cnt += 1
        mat = pivoting(mat, i, m)

    x = backsubstitution(mat) # this `x` contains the value of solutions (eg- x0,x1,x2...)

    print(f"final matrix after partial pivoting:\n{mat}")
    print(f"Number of row interchanges: {cnt}")
    print(f"SOLUTIONS: {x}")

    
#function to do pivoting
def pivoting(mat, target_col, total_row):
    # mat = scale_row(mat, 1/mat[target_col][target_col],target_col)
    for i in range(target_col + 1, total_row):
        m = mat[i][target_col]/mat[target_col][target_col]
        mat = add_rows(mat, -1 * m, i, target_col)
    return mat

#function for backsubstitution and it will return a list of solutions 
def backsubstitution(mat):
    shape = mat.shape
    m = shape[0]
    n = shape[1]

    x = [0] * m 
    x[m-1] = mat[m-1][n-1] / mat[m-1][m-1] 
    for i in range(m-2,-1,-1):
        ax = 0
        for j in range (i+1, n-1):
            ax += mat[i][j] * x[j]
        x[i] = (mat[i][n-1] - ax) / mat[i][i]

    return x



partial_pivoting_method(np.array([[2, -1, 1, 8], [3, 6, 9, 2], [3, 3, 5, 1]]))

final matrix after partial pivoting:
[[3.         3.         5.         1.        ]
 [0.         3.         4.         1.        ]
 [0.         0.         1.66666667 8.33333333]]
Number of row interchanges: 1
SOLUTIONS: [-1.6666666666666667, -6.333333333333331, 4.999999999999998]


In [65]:
def gauss_jordan_elimination(mat):
    mat = mat.astype(float)
    m, n = np.shape(mat)
    I = np.eye(m, dtype=float)

    augmented_matrix = np.hstack((mat, I))

    i = 0
    while i < m:
        # Pivot the current row
        pivot = augmented_matrix[i, i]
        j = 0
        while j < m + n:
            augmented_matrix[i, j] /= pivot
            j += 1

        # Eliminate other rows
        j = 0
        while j < m:
            if i != j:
                factor = augmented_matrix[j, i]
                k = 0
                while k < m + n:
                    augmented_matrix[j, k] -= factor * augmented_matrix[i, k]
                    k += 1
            j += 1
        i += 1

    return augmented_matrix

# Example usage:
Y = np.array([[5, 4, 2], [-1, 2, 1], [1, 1, 1]])

# Compute the augmented matrix using the simplified function
augmented_matrix = gauss_jordan_elimination(Y)

# Display the results
print("Original Matrix:")
print(np.round(Y, 2))
print("\nInversed Augmented Matrix:")
print(np.round(augmented_matrix, 2))

Original Matrix:
[[ 5  4  2]
 [-1  2  1]
 [ 1  1  1]]

Inversed Augmented Matrix:
[[ 1.    0.    0.    0.14 -0.29  0.  ]
 [ 0.    1.    0.    0.29  0.43 -1.  ]
 [ 0.    0.    1.   -0.43 -0.14  2.  ]]


In [ ]:
def jacobi_method(A,b,max_iter,Tolerance):
    n = np.size(A[0])
    b = np.reshape(b,[n,1])
    z = np.reshape(np.diagonal(A),[n,1])
    # i != j
    B = A - np.diag(np.diagonal(A))
    x_o = np.zeros_like(b)
    for i in range(max_iter):
        x = (b - np.dot(B,x_o))/z
        e = np.linalg.norm(x-x_o, ord='fro')
        print(x,e)
        if e < Tolerance:
            print(f'Iterations converged after {i} steps')
            break
        x_o = x
    return x
    

Solution: [4.91071403 4.64285687 3.66071403]


In [ ]:
import numpy as np
from scipy.linalg import norm
def myGaussSeidel(A,b,max_iter,Tolerance):
    n = np.size(A[0])
    b = np.reshape(b,[n,1])
    z = np.reshape(np.diagonal(A),[n,1])
    x   = np.zeros_like(b,dtype=np.double)
    for i in range(max_iter):
        x_o = x.copy()
        for j in range(n):
            x[j] = b[j] - np.dot(A[j,:j],x[:j])- np.dot(A[j,j+1:],x_o[j+1:])
            x[j] = x[j]/A[j,j]
        print(f'{norm(x-x_o)}')    
        if norm(x-x_o) < Tolerance:
                print(f'Iterations converged after {i} steps')
                print(f'{(x-x_o)}')
                break
    return x

Solution: [4.91071414 4.64285707 3.66071427]


In [ ]:
# Integration (Monte Carlo)

In [75]:
# Integration (Area)
def integration_trape(x):
    return x**2 + 5*x + 5

a = 6
b = 10
dx = 0.001
integral = 0

for i in range(abs(b-a)*1000):
    c = a + dx
    avg = (integration_trape(a)+integration_trape(c))/2
    integral += avg*dx
    a += dx
    
print(integral)

441.33333400001834


In [1]:
# LU Method

import numpy as np

def lu(A):
    n = A.shape[0]
    
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    for i in range(n):
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] = U[i+1:] - factor[:, np.newaxis] * U[i]
        
    return L, U

# Example usage:
AA = np.array([[2, -1, 1], [3, 3, 9], [3, 3, 5]])
b = np.array([8, 3, 1])

L, U = lu(AA)
L1 = L @ U
y = np.linalg.solve(L1, b.T)
x = np.linalg.solve(U, y)

print("L:")
print(L)
print("\nU:")
print(U)
print("\nL1 (L times U):")
print(L1)
print("\nSolution (x):")
print(x)


L:
[[1.  0.  0. ]
 [1.5 1.  0. ]
 [1.5 1.  1. ]]

U:
[[ 2 -1  1]
 [ 0  4  7]
 [ 0  0 -4]]

L1 (L times U):
[[ 2.  -1.   1. ]
 [ 3.   2.5  8.5]
 [ 3.   2.5  4.5]]

Solution (x):
[ 0.875  -0.5625 -0.125 ]
